In [1]:
from uk_covid19 import Cov19API
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wdg
from IPython.display import clear_output

In [ ]:
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Data Source: Public Health England API

In [ ]:
# We are going to develop a simple dashboard which displays the graph of 
# a comparison of the number of new cases with the number tests, broke down by region.
# It will allow users to select regions and get the corresponding output in real-time.

In [ ]:
# Section 1 is to load and wrangle the data

In [ ]:
# define filters to get COVID-19 data for different regions
filters = [
    'areaType=region'
]

# define the structure of the data to retrieve
# values here are the names of the PHE metrics
structure = {
    "date": "date",
    "areaName": "areaName",
    "newCases": "newCasesBySpecimenDate",
    "newTests": "newVirusTestsBySpecimenDate"
}

In [ ]:
api = Cov19API(filters=filters, structure=structure)

In [ ]:
# retrieve COVID-19 data from the API in JSON format
covid_data=api.get_json()

In [8]:
# save the retrieved data to a JSON file
with open("covid_data.json", "wt") as OUTF:
    json.dump(covid_data, OUTF)

In [9]:
# enable the embedding of matplotlib output
%matplotlib inline

In [10]:
# get the saved COVID-19 data from the JSON file
with open("covid_data.json", "rt") as INFILE:
    data=json.load(INFILE)

In [11]:
# convert the data to a DataFrame
df=pd.DataFrame(data["data"])
df["date"]=pd.to_datetime(df["date"])

In [12]:
# use Pandas pivot function to reshape the DataFrame
pivot_data = df.pivot_table(
    index="date",
    columns="areaName",
    values=["newCases", "newTests"],
    aggfunc='sum'
)

pivot_data = pivot_data.reset_index()

# join the column names
new_columns = ['_'.join(col).strip() for col in pivot_data.columns.values]

# assign the new column names to the DataFrame
pivot_data.columns = new_columns

In [13]:
# save the DataFrame to a pickle file for later use
df.to_pickle("df.pkl")

In [14]:
# read the saved DataFrame from the pickle file
df=pd.read_pickle("df.pkl")

In [15]:
# Section 2 is about data visualization and 
# adding interactive controls for the dashboard.

In [16]:
# The button below allows users to refresh data from the Public Health England API.

In [17]:
def access_api(button):
    apibutton.icon="check"
    apibutton.disabled=True

apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='download'
)

apibutton.on_click(access_api)

In [18]:
# The selection checkboxes allow users to select regions of interest for comparison.

In [19]:
# get a list of region names
regions = df['areaName'].unique()

# create Checkbox widgets for each region
checkboxes = [
    wdg.Checkbox(description=region) 
    for region in regions
]

output = wdg.Output()

# create a VBox containing a HBox to organize the checkbox widgets
vbox = wdg.VBox(
    children=[
        wdg.HBox(children=checkboxes),
        output
    ]
)

# define the function to update the plot data based on the selected regions
def update_plot(**kwargs): #since we have a large number of checkboxes...
    with output:  
        clear_output(wait=True)  # clear previous output
        selected_regions = [checkbox.description for checkbox in checkboxes if checkbox.value]

        plt.figure(figsize=(15, 6))
        
        for region in selected_regions:
            region_data = df[df['areaName'] == region].dropna(subset=['newCases'])
            plt.plot(region_data['date'], region_data['newCases'], label=f'Cases - {region}')
            plt.plot(region_data['date'], region_data['newTests'], label=f'Tests - {region}', linestyle='dashed')

        plt.title('Covid-19 Cases and Tests Comparison')
        plt.xlabel('Date')
        plt.ylabel('Count')
        if plt.gca().get_lines(): 
            plt.legend(loc='upper left', fontsize='medium') 
        plt.show()
        
# attach the update_plot function to the button's 'on_click' event
apibutton.on_click(access_api)

# create an interactive_output widget
interactive_output = wdg.interactive_output(
    update_plot, 
    {f'checkbox_{i}': checkbox for i, checkbox in enumerate(checkboxes)}
)

description_text = '''
    ## COVID-19 Data Dashboard
    
    This dashboard allows you to visualize the comparison of Covid-19 cases and tests for different regions.
    
    **Data Source:**
    
    The data is sourced from the Public Health England API, providing up-to-date information on Covid-19 cases and tests for various regions.
    
    **Instructions:**
    
    1. Click the "Refresh data" button to download the latest data from the Public Health England API.
    2. Use the checkboxes to select regions for comparison.
    3. The plot will dynamically update based on your region selection.
'''

display(description_text, apibutton, vbox, interactive_output)

'\n    ## COVID-19 Data Dashboard\n    \n    This dashboard allows you to visualize the comparison of Covid-19 cases and tests for different regions.\n    \n    **Data Source:**\n    \n    The data is sourced from the Public Health England API, providing up-to-date information on Covid-19 cases and tests for various regions.\n    \n    **Instructions:**\n    \n    1. Click the "Refresh data" button to download the latest data from the Public Health England API.\n    2. Use the checkboxes to select regions for comparison.\n    3. The plot will dynamically update based on your region selection.\n'

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

Output()

In [ ]:
!voila dashboard.ipynb

[Voila] Using /tmp to store connection files
[Voila] Storing connection files in /tmp/voila_17wlxmnz.
[Voila] Serving static files from /opt/conda/lib/python3.10/site-packages/voila/static.
[Voila] Voilà is running at:
http://localhost:8866/
[Voila] WARNING | No web browser found: could not locate runnable browser.
